In [2]:
import pandas as pd
import pyodbc
import os 
import sqlalchemy
import getpass
from sqlalchemy import create_engine, inspect, text
from sqlalchemy.exc import SQLAlchemyError
from utils.sql import sql_col_type, transform_to_sql_server, merge_from_folders, run_procedure


In [2]:
fact_lailo = merge_from_folders(
                r"C:\Users\Admin\OneDrive - ueh.edu.vn\Documents\Python\DemoLailo\CNX_Fake", 
                r"C:\Users\Admin\OneDrive - ueh.edu.vn\Documents\Python\DemoLailo\HST_Fake" 
                )
fact_lailo

,Mã hàng hóa,Tên hàng hóa,Nhóm HHDV,ĐVT,Số lượng,Doanh thu,Các khoản giảm trừ,Doanh thu thuần,Giá vốn,Lợi nhuận,Year Month,Công ty,SourceFile
0,14-20-081,Nước Hồng Trà Đào Tiến Thành (450ml*24chai),5.Sản phẩm nước,Thùng,16795,2383747940,6.319109e+07,2.320557e+09,141932,-227521863,2023-01,CNX,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...
1,DV-001,Dịch vụ bốc xếp,9.HHDV khác,Tấn,77820,5049428520,2.531023e+08,4.796326e+09,64886,-529389014,2023-01,CNX,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...
2,DV-004,Cho thuê xe,9.HHDV khác,Tháng,38194,3723838612,3.275334e+08,3.396305e+09,97498,-534285667,2023-01,CNX,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...
3,15-02-003,Skimmilk Powder LH 1x25kg( sữa bột gầy LH),8.Hàng Hóa NL,KG,61263,1594247049,1.093334e+08,1.484914e+09,26023,-342285876,2023-01,CNX,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...
4,15-02-008,Mfitl-P (S) (Precipitated Silica - Silicon Dio...,8.Hàng Hóa NL,KG,1769,123361215,1.047581e+07,1.128854e+08,69735,-103344769,2023-01,CNX,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4927,TR-015,Trà hòa tan Tiến Thành- Hương Chanh (20hộp*24g...,1.SP Trà hoà tan,Thùng,46706,8588719634,1.949536e+08,8.393766e+09,183889,2540727710,2025-7,HST,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...
4928,TR-016,Trà hòa tan Tiến Thành- Hương Dâu (20hộp*24g*1...,1.SP Trà hoà tan,Thùng,51799,6712425214,4.573909e+08,6.255034e+09,129586,3530074606,2025-7,HST,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...
4929,TR-018,Bột hòa tan Tiến Thành- Hương Sâm dứa sữa (20 ...,1.SP Trà hoà tan,Thùng,98379,1804201073,1.364539e+08,1.667747e+09,119987,2818417369,2025-7,HST,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...
4930,TR-019,Trà hòa tan Tiến Thành- Hương Me (20 hộp*24g*1...,1.SP Trà hoà tan,Thùng,42048,2108454912,6.212406e+07,2.046331e+09,50144,3202537347,2025-7,HST,C:\Users\Admin\OneDrive - ueh.edu.vn\Documents...


In [3]:
# Đẩy FACT vào LailoStaging
transform_to_sql_server(
    df=fact_lailo,
    server="LAPTOP-UVA5Q9QT",
    database_name="LailoStaging",
    schema_name="FACT",      
    table_name="Lailo",     
    if_exist="replace"        # replace xóa tạo lại
)

==> Loading 4932 rows vào [FACT].[Lailo] ...
==> Done!


In [ ]:
# Đẩy DIM vào LailoStaging
dim_files = {
    "Date": r"C:\Users\Admin\OneDrive - ueh.edu.vn\Documents\Python\DemoLailo\DIM\Date.xlsx",
    "Product": r"C:\Users\Admin\OneDrive - ueh.edu.vn\Documents\Python\DemoLailo\DIM\Products.xlsx",
    "Company": r"C:\Users\Admin\OneDrive - ueh.edu.vn\Documents\Python\DemoLailo\DIM\Company.xlsx"
}
for table, path in dim_files.items():
    df = pd.read_excel(path)   # chỉ việc đọc file Excel
    df["SourceFile"] = path
    transform_to_sql_server(
        df=df,
        server="LAPTOP-UVA5Q9QT",
        database_name="LailoStaging",
        schema_name="DIM",      
        table_name= table,     
        if_exist="replace" 
    )


==> Loading 48 rows vào [DIM].[Date] ...
==> Done!
==> Loading 570 rows vào [DIM].[Product] ...
==> Done!
==> Loading 2 rows vào [DIM].[Company] ...
==> Done!


In [ ]:
# Chạy procedure từ LailoStaging vào LailoDWH
server = "LAPTOP-UVA5Q9QT"
database = "LailoDWH"
procs = [
    "DIM.ETL_Load_DimDate",
    "DIM.ETL_Load_DimProduct",
    "DIM.ETL_Load_DimCompany",
    "FACT.ETL_Load_FactLailo"
]
for p in procs:
    run_procedure(server, database, p)

==> Executed procedure: DIM.ETL_Load_DimDate
==> Executed procedure: DIM.ETL_Load_DimProduct
==> Executed procedure: DIM.ETL_Load_DimCompany
==> Executed procedure: FACT.ETL_Load_FactLailo
